In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.stats import norm
from ploting import *
import seaborn as sns
%matplotlib inline

import torch
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
torch.manual_seed(123)

## Create dataset

In [27]:
def generate_data(samples=500000):
    data = []
    labels = []
    for i in range(samples):
        category = (np.random.choice([0, 1]), np.random.choice([0, 1]))
        if category == (1, 0):
            data.append([np.random.uniform(0.1, 1), 0])
            labels.append([1, 0, 1])
        if category == (0, 1):
            data.append([0, np.random.uniform(0.1, 1)])
            labels.append([0, 1, 0])
        if category == (0, 0):
            data.append([np.random.uniform(0.1, 1), np.random.uniform(0.1, 1)])
            labels.append([0, 0, 1])
    return train_test_split(data, labels, random_state=42)      

In [28]:
X_train, X_test, y_train, y_test = generate_data()

In [32]:
np.hstack(y_train).shape

(844509,)

## Define model

In [10]:
class Classifier(torch.nn.Module):

    def __init__(self, nb_label):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Classifier, self).__init__()
        self.main = torch.nn.Sequential(torch.nn.Linear(2, 64),
                                  torch.nn.ReLU(),
                                  torch.nn.Linear(64, nlabel),)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        
        return self.main(x)
        

In [16]:
def model_train(x_data, y_data, model, lrate, epochs=10000):
    
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lrate)
    criterion = torch.nn.MultiLabelSoftMarginLoss()
    
    Epoch = []
    Loss  = []
   
    for epoch in range(epochs):
        losses = []
        for i, sample in enumerate(x_data):
            x_data = Variable(torch.FloatTensor(sample)).view(1, -1)
            y_data = Variable(torch.FloatTensor(y_data[i])).view(1, -1)
            
            if torch.cuda.is_available():
                model = model.cuda()
                x_data = x_data.cuda()
                y_data = y_data.cuda()
            
            y_pred = model(x_data)
            loss = criterion(y_pred, y_data)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            
            losses.append(loss.data.mean())
        Epoch.append(epoch)  
        Loss.append(np.mean(losses))
        if (epoch % 500 == 0):
            print('[%d/%d] Loss: %.3f' % (epoch+1, epochs, np.mean(losses)))
    
    return Epoch, Loss 

In [17]:
nlabel = len(y_train[0])
classifier = Classifier(nlabel)
print(classifier)

Classifier(
  (main): Sequential(
    (0): Linear(in_features=2, out_features=64)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=3)
  )
)


In [18]:
epoch, loss = model_train(X_train, y_train, classifier, 0.01)

IndexError: index 1 is out of range for dimension 0 (of size 1)